# 원문

In [5]:
train_df['Text'][0]

'NH농협금융지주는 김광수 회장이 22일 서대문  본사에서 농협금융 자회사 대표들과 3분기 종합경영성과 분석회의를 했다고 23일 밝혔다. \n회의에서는 NH농협카드와 NH농협캐피탈의 체질개선과 질적 성장 방안이 주로 논의됐다. \n김 회장은 농협카드에 "조직 효율성과 인적 전문성을 높이고, 독립적  성과평가를 통해 책임경영 체제를 확립하라"고 주문했다. \n또 "상품별·고객별 수익성 분석을 고도화해 일반 카드사 수준의 경영 인프라를 구축하라"고 당부했다. \n김 회장은 NH농협캐피탈과 관련해서는 산업재 금융은 건설경기 악화를 염두에  두고 자산 건전성 개선 노력을 이어가고, 기업금융은 취급시부터 위험관리를 철저히 해야 한다고 강조했다. \n김 회장은 연말 인사에서는 업무경력과 직무 전문성을 우선 고려하겠다고  말했다.\n[연합뉴스] Copyrights ⓒ 연합뉴스. 무단 전재 및 재배포금지\n\nwindow.jQuery || document.write("<script src=\'//common.mk.co.kr/common/include/mklib/jquery.js\'>")\n\n'

# 정규표현식 개선 전 (100개 문서 전처리)

In [6]:
%%time
' '.join(nlp.extract_tokens_for_all_document_FAST_VERSION(train_df['Text'].iloc[:100], if_morphs = True)[0])

CPU times: user 18.5 s, sys: 403 ms, total: 18.9 s
Wall time: 3.78 s


'NH 농협 금융 지주 김광수 회장 22일 서대문 본사 에서 농협 금융 자회사 대표 3분 종합 경영 성과 분석 회의 했다고 23일 밝혔다 회의 에서는 NH 농협카드 NH 농협 캐피탈 체질개선 질적 성장 방안 주로 논의 됐다 회장 농협카드 조직 효율 성과 인적 전문성 높이 독립 성과 평가 통해 책임 경영 체제 확립 라고 주문 했다 상품 고객 수익 분석 고도화 일반 카드 수준 경영 인프라 축하 라고 당부 했다 회장 NH 농협 캐피탈 관련 해서는 산업 금융 건설 경기 악화 염두 두고 자산 건전성 개선 노력 어가 기업 금융 취급 부터 위험관리 철저히 해야 한다고 강조 했다 회장 연말 인사 에서는 업무 경력 직무 전문성 우선 고려 하겠다고 했다'

# 정규표현식 개선 후 (100개 문서 전처리)

In [9]:
%%time
' '.join(nlp.extract_tokens_for_all_document_FAST_VERSION(train_df['Text'].iloc[:100], if_morphs = True)[0])

CPU times: user 2.95 s, sys: 69.5 ms, total: 3.02 s
Wall time: 773 ms


'농협 금융 지주 김광수 회장 서대문 본사 에서 농협 금융 자회사 대표 종합 경영 성과 분석 회의 했다고 밝혔다 회의 에서는 농협카드 농협 캐피탈 체질개선 질적 성장 방안 주로 논의 됐다 회장 농협카드 조직 효율 성과 인적 전문성 높이 독립 성과 평가 통해 책임 경영 체제 확립 라고 주문 했다 상품 고객 수익 분석 고도화 일반 카드 수준 경영 인프라 축하 라고 당부 했다 회장 농협 캐피탈 관련 해서는 산업 금융 건설 경기 악화 염두 두고 자산 건전성 개선 노력 어가 기업 금융 취급 부터 위험관리 철저히 해야 한다고 강조 했다 회장 연말 인사 에서는 업무 경력 직무 전문성 우선 고려 하겠다고 했다'

## **제목만 있고 특수 문자를 제거하면 기사 내용이 전혀 없는 뉴스도 존재**

### 약 9만개 데이터 중, 2124개 (약 2.5%)기사가 아무런 내용이 없음.

In [5]:
len(none_idx_ls)

2124

### 뉴스 기사에 토큰이 하나 밖에 없는 경우도 존재함

In [13]:
train_df['Num of Tokens'].describe()

count    87876.000000
mean       209.709921
std        157.452864
min          1.000000
25%        112.000000
50%        176.000000
75%        268.000000
max       4327.000000
Name: Num of Tokens, dtype: float64

### 토큰의 수가 50개 이하인 기사는 전체의 5.5% 수준

In [37]:
np.sum([train_df['Num of Tokens'] <= 50]) / len(train_df)

0.055908325367563384

### 토큰이 50개인 기사의 길이는 대략 2줄 정도

In [27]:
' '.join(train_df[train_df['Num of Tokens'] == 50].iloc[100].tolist()[0])

'농협 은행 추석 연휴 맞아 현금 유가 증권 귀중 이용 수수료 없이 보관 해주는 서비스 운영 한다고 밝혔다 서비스 오는 전국 영업 에서 실시 한다 가까운 영업 금고 남아있는지와 보호 예수 절차 문의 하고 이용 하면 된다 자세한 내용 농협 인터넷 뱅킹 홈페이지 영업 고객 행복 센터 문의 하면 된다'

## 토큰의 수가 50개 이하인 기사는 표본에서 제거

### 각 섹션 별로 남은 기사의 비율 체크

체크한 결과 경제 기사가 문제가 많았던 것으로 확인 됨,

약 20%의 기사가 제거됨..

In [39]:
Counter(train_df['Section'])

Counter({'IT과학': 9836,
         '경제': 8050,
         '국제': 9677,
         '기업': 9604,
         '문화': 9313,
         '부동산': 9714,
         '사회': 9325,
         '정치': 8875,
         '증권': 8569})